# Tools

In [1]:
import os, numpy as np, glob
import mcvine
from sampleassembly.crystal.ioutils import xyzfile2unitcell, ciffile2unitcell
from sampleassembly.visitors.CrossSectionCalculator import CrossSectionCalculator
from mcvine.workflow.sampleassembly import scaffolding
from mcvine.workflow.sample import loadSampleYml
from matplotlib import pyplot as plt
%matplotlib notebook

# Al crystal

In [2]:
structure = xyzfile2unitcell('Al.xyz')
# structure = ciffile2unitcell('Al_mp-134_symmetrized.cif')

In [3]:
xscalc = CrossSectionCalculator()
xscalc.include_density = False
print("abs (2200m/s), inc, coh cross sections = ", xscalc.onUnitCell(structure))
xscalc.include_density = True
print("abs (2200m/s), inc, coh inverse length =", xscalc.onUnitCell(structure))

abs (2200m/s), inc, coh cross sections =  [2.31e-29*m**2 8.2e-31*m**2 1.495e-28*m**2]
abs (2200m/s), inc, coh inverse length = [1.39164*m**-1 0.0494001*m**-1 9.00648*m**-1]


# Sample with Powder Diffraction Kernel

In [4]:
!rm -rf Al-powder-diffraction
!mkdir -p Al-powder-diffraction

## Peaks python file

* If you have a laz file, you may want to use that.
* Shown here is a quick way to create diffraction peaks from crystal structure

In [5]:
from mccomponents.sample import diffraction, matter

In [6]:
structure = matter.loadCif('Al_mp-134_symmetrized.cif')
diffraction.create_peaks_py('Al-diffr-peaks.py', structure, 300., max_index=20, min_dspacing=.5)

/home/97n/dv/mcvine/export-3.7/lib/python3.7/site-packages/mccomponents/sample/diffraction/calcpeaks.py:85: RuntimeWarning: divide by zero encountered in double_scalars
  return 1./np.linalg.norm(recvec)


In [7]:
!head -n 10 Al-diffr-peaks.py

from mccomponents.sample.diffraction.SimplePowderDiffractionKernel import Peak

peaks = [
    Peak(hkl=(1, 1, 1), d=2.331877322738055, q=2.6944750677544076, F=(13.336929215115559-2.4499541510619136e-15j), F_squared=1.778736808890029, multiplicity=8, intrinsic_line_width=0, DebyeWaller_factor=0),
    Peak(hkl=(0, 0, 2), d=2.019465, q=3.1113118113854843, F=(13.187325715565134-1.614981622688042e-15j), F_squared=1.7390555952840545, multiplicity=6, intrinsic_line_width=0, DebyeWaller_factor=0),
    Peak(hkl=(0, 2, 2), d=1.427977395868891, q=4.400059360432953, F=(12.605505909568386-3.0874584927571937e-15j), F_squared=1.5889877923616351, multiplicity=12, intrinsic_line_width=0, DebyeWaller_factor=0),
    Peak(hkl=(1, 1, 3), d=1.2177832149554666, q=5.15952694208333, F=(12.186049582243726-3.73090165378643e-15j), F_squared=1.4849980442090247, multiplicity=24, intrinsic_line_width=0, DebyeWaller_factor=0),
    Peak(hkl=(2, 2, 2), d=1.1659386613690275, q=5.388950135508815, F=(12.04935577261231-4.4

## Yaml spec

In [8]:
%%file Al-powder-diffraction.yaml
name: Al
# structure_file: Al_mp-134_symmetrized.cif
structure_file: Al.xyz
excitation:
 type: powder_diffraction
 Dd_over_d: 1e-5
 peaks_py_path: Al-diffr-peaks.py
shape: cylinder radius="1*mm" height="3*mm"
temperature: 300*K

Overwriting Al-powder-diffraction.yaml


## Convert to xml

In [9]:
spec = loadSampleYml('Al-powder-diffraction.yaml')
scaffolding.createSampleAssembly('Al-powder-diffraction', spec, add_elastic_line=False)

Copying Al-diffr-peaks.py to Al-powder-diffraction/


/home/97n/dv/mcvine/export-3.7/lib/python3.7/site-packages/mcvine/workflow/sampleassembly/scaffolding/sampleassembly_xml.py:22: UserWarning: assume average temperature
  warnings.warn("assume average temperature")


In [10]:
ls Al-powder-diffraction/

Al-diffr-peaks.py  Al-scatterer.xml  Al.xyz  sampleassembly.xml


In [11]:
!mcvine sampleassembly check Al-powder-diffraction//sampleassembly.xml

weight:  1.0
